# Gemini PDF worker

In [16]:
# ─── Imports ────────────────────────────────────────────────────────────────
import json, base64, os
from dotenv import load_dotenv
from pypdf import PdfReader
from google import genai
from google.genai import types

In [17]:
# ─── Setup ────────────────────────────────────────────────────────────────
MODEL = "gemini-2.5-flash-preview-04-17"

load_dotenv(dotenv_path=".env.local")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=GEMINI_API_KEY)

In [18]:
# ─── Helpers ──────────────────────────────────────────────────────────────
def extract_fields_with_coords(pdf_path):
    reader = PdfReader(pdf_path)
    fields = []
    for page_num, page in enumerate(reader.pages, start=1):
        for annot in page.get("/Annots", []):
            obj = annot.get_object()
            name = obj.get("/T")
            rect = obj.get("/Rect")
            if name and rect:
                x1, y1, x2, y2 = rect
                fields.append({
                    "field_id": name,
                    "page":     page_num,
                    "coords":   [x1, y1, x2, y2],
                })
    return fields

In [19]:
# ─── Real Gemini Call ─────────────────────────────────────────────────────
def enrich_fields_with_gemini(raw_fields, instructions_path, language="en"):
    # 1) Read & encode instructions PDF
    with open(instructions_path, "rb") as f:
        inst_b64 = base64.b64encode(f.read()).decode("utf-8")

    # 2) Build your prompt (JSON-in, JSON-out)
    prompt = f"""
I have a USCIS form with these fields (field_id, page, coords):
{json.dumps(raw_fields, indent=2)}

I also have the form instructions PDF encoded in base64:
{inst_b64}

Please, for each field, output a JSON object with keys:
  - field_id
  - label           (short human label)
  - page
  - coords
  - dependencies    (list of other field_ids)
  - gemini_note     (plain-English explanation)
  - examples        (list with one example)
Return strictly valid JSON in the format: {{ "fields": [ ... ] }}.
Language: {language}
"""

    # 3) Call Gemini
    response = client.models.generate_content(
        model=MODEL,
        contents=[
            types.Content(
                role="user", 
                parts=[ 
                    types.Part.from_text(text=prompt) 
                ]
            )
        ],
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0),
            response_mime_type="application/json",
        )
    )

    # 4) Parse & return
    result = json.loads(response.text)
    return result["fields"]

In [20]:
form_pdf = "static/forms/I-765/form.pdf"
inst_pdf = "static/forms/I-765/inst.pdf"

raw = extract_fields_with_coords(form_pdf)
enriched = enrich_fields_with_gemini(raw, inst_pdf, language="en")

print(json.dumps(raw))

#print(json.dumps(enriched, indent=2))

[{"field_id": "PDF417BarCode1[0]", "page": 1, "coords": [191.999, 11.999, 461.999, 29.999]}, {"field_id": "Line1a_FamilyName[0]", "page": 1, "coords": [120.002, 132.001, 294.001, 150.001]}, {"field_id": "Line1b_GivenName[0]", "page": 1, "coords": [120.002, 108.006, 294.001, 126.006]}, {"field_id": "Line1c_MiddleName[0]", "page": 1, "coords": [120.002, 84.005, 294.001, 102.005]}, {"field_id": "Part1_Checkbox[0]", "page": 1, "coords": [61.0, 409.002, 71.0, 419.002]}, {"field_id": "Part1_Checkbox[1]", "page": 1, "coords": [61.0, 391.002, 71.0, 401.002]}, {"field_id": "Part1_Checkbox[2]", "page": 1, "coords": [61.0, 246.999, 71.0, 256.999]}, {"field_id": "CheckBox1[0]", "page": 1, "coords": [246.501, 529.001, 256.501, 539.001]}, {"field_id": "Line2a_FamilyName[0]", "page": 1, "coords": [402.001, 366.001, 576.0, 384.001]}, {"field_id": "Line2b_GivenName[0]", "page": 1, "coords": [402.001, 342.006, 576.0, 360.006]}, {"field_id": "Line2c_MiddleName[0]", "page": 1, "coords": [402.001, 318.005,

In [21]:
import json
import uuid
import os
from pathlib import Path
from typing import Dict, Optional
from pypdf import PdfReader, PdfWriter
from pypdf.generic import NameObject, TextStringObject, BooleanObject

# Configuration (adjust paths to match your setup)
STATIC_FORMS_DIR = Path("static/forms")
OUTPUT_DIR = Path("output/filled_pdfs")
BASE_URL = "http://localhost:8000"

# Create output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"✓ Setup complete. Found {len(raw)} fields to work with.")
print(f"Sample field: {raw[0]}")

✓ Setup complete. Found 170 fields to work with.
Sample field: {'field_id': 'PDF417BarCode1[0]', 'page': 1, 'coords': [191.999, 11.999, 461.999, 29.999]}


In [22]:
def fill_pdf_form(input_pdf_path: str, output_pdf_path: str, field_data: Dict[str, str]) -> bool:
    """
    Fill PDF form fields with provided data using pypdf.
    """
    try:
        reader = PdfReader(input_pdf_path)
        writer = PdfWriter()

        # Process each page
        for page_num, page in enumerate(reader.pages):
            # Check if page has annotations (form fields)
            if "/Annots" in page:
                annotations = page["/Annots"]

                # Process each annotation
                for annot in annotations:
                    field = annot.get_object()

                    # Check if this is a form field with a name
                    if "/T" in field:
                        field_name = field["/T"]

                        # If we have data for this field, fill it
                        if field_name in field_data:
                            field_value = field_data[field_name]
                            field_type = field.get("/FT")

                            # Handle different field types
                            if field_type == "/Tx":  # Text field
                                field.update({
                                    NameObject("/V"): TextStringObject(field_value)
                                })
                            elif field_type == "/Ch":  # Choice field (dropdown/listbox)
                                field.update({
                                    NameObject("/V"): TextStringObject(field_value)
                                })
                            elif field_type == "/Btn":  # Button field (checkbox/radio)
                                # For checkboxes, expect "true"/"false" or "yes"/"no"
                                is_checked = field_value.lower() in ["true", "yes", "1", "on"]
                                if is_checked:
                                    # Find the "Yes" value for this checkbox
                                    if "/AP" in field and "/N" in field["/AP"]:
                                        ap_dict = field["/AP"]["/N"]
                                        if isinstance(ap_dict, dict):
                                            # Usually checkboxes have keys like "Yes", "Off", etc.
                                            yes_keys = [k for k in ap_dict.keys() if k != "/Off"]
                                            if yes_keys:
                                                field.update({
                                                    NameObject("/V"): NameObject(yes_keys[0]),
                                                    NameObject("/AS"): NameObject(yes_keys[0])
                                                })
                                else:
                                    field.update({
                                        NameObject("/V"): NameObject("/Off"),
                                        NameObject("/AS"): NameObject("/Off")
                                    })

            # Add the page to the writer
            writer.add_page(page)

        # Copy form information
        if "/AcroForm" in reader.trailer["/Root"]:
            writer._root_object.update({
                NameObject("/AcroForm"): reader.trailer["/Root"]["/AcroForm"]
            })

        # Write the filled PDF
        with open(output_pdf_path, "wb") as output_file:
            writer.write(output_file)

        return True

    except Exception as e:
        print(f"Error filling PDF: {e}")
        return False

# Mock sessions for testing (adjust to match your session structure)
_MOCK_SESSIONS = {
    "test_session_123": {
        "form_name": "form.pdf",  # Update this to your actual PDF filename
        "form_type": "i-765",
        "required_fields": ["Line1a_FamilyName[0]", "Line1b_GivenName[0]", "Line19_DOB[0]"],
        "all_fields": [field["field_id"] for field in raw[:10]],  # Use first 10 fields
        "status": "active"
    }
}

def validate_required_fields(session_id: str, answers: Dict[str, str]) -> Dict[str, any]:
    """Test the validation function with your actual field data."""
    if session_id not in _MOCK_SESSIONS:
        return {"valid": False, "error": "Session not found"}

    session_info = _MOCK_SESSIONS[session_id]
    required_fields = session_info.get("required_fields", [])

    missing_fields = [field for field in required_fields if field not in answers or not answers[field].strip()]

    completion_percentage = len(answers) / len(session_info.get("all_fields", [])) * 100 if session_info.get("all_fields") else 0

    return {
        "valid": len(missing_fields) == 0,
        "missing_fields": missing_fields,
        "completion_percentage": round(completion_percentage, 2),
        "total_fields": len(session_info.get("all_fields", [])),
        "filled_fields": len(answers)
    }

def get_form_path(session_id: str, sessions_data: dict) -> Optional[str]:
    """Get path to your actual PDF form."""
    if session_id not in sessions_data:
        return None

    session_info = sessions_data[session_id]
    form_name = session_info.get("form_name")  # e.g., "form.pdf"

    if form_name:
        # Update this path to match where your PDF is located
        form_path = Path("static/forms/I-765") / form_name  # Adjust path as needed
        if form_path.exists():
            return str(form_path)

    return None

print("✓ Functions loaded successfully")

✓ Functions loaded successfully


In [23]:
print("=== Testing Field Data Analysis ===")

# Analyze your extracted fields
field_types = {}
pages_with_fields = {}

for field in raw:
    field_id = field["field_id"]
    page = field["page"]
    
    # Categorize by type (based on naming patterns)
    if "Checkbox" in field_id:
        field_type = "checkbox"
    elif "BarCode" in field_id:
        field_type = "barcode"
    elif any(name in field_id for name in ["FamilyName", "GivenName", "MiddleName"]):
        field_type = "name"
    elif "DOB" in field_id or "Date" in field_id:
        field_type = "date"
    elif "Number" in field_id:
        field_type = "number"
    elif "Address" in field_id or "Street" in field_id or "City" in field_id:
        field_type = "address"
    else:
        field_type = "text"
    
    field_types[field_type] = field_types.get(field_type, 0) + 1
    pages_with_fields[page] = pages_with_fields.get(page, 0) + 1

print("Field Type Analysis:")
for field_type, count in field_types.items():
    print(f"  {field_type}: {count} fields")

print(f"\nFields by Page:")
for page, count in sorted(pages_with_fields.items()):
    print(f"  Page {page}: {count} fields")

print(f"\nTotal fields extracted: {len(raw)}")


=== Testing Field Data Analysis ===
Field Type Analysis:
  barcode: 7 fields
  name: 23 fields
  checkbox: 27 fields
  text: 65 fields
  number: 36 fields
  address: 6 fields
  date: 6 fields

Fields by Page:
  Page 1: 18 fields
  Page 2: 43 fields
  Page 3: 27 fields
  Page 4: 15 fields
  Page 5: 35 fields
  Page 6: 7 fields
  Page 7: 25 fields

Total fields extracted: 170


In [24]:
print("\n=== Testing Validation Function ===")

# Test case 1: Complete data
complete_answers = {
    "Line1a_FamilyName[0]": "Rodriguez",
    "Line1b_GivenName[0]": "Maria",
    "Line1c_MiddleName[0]": "Elena",
    "Line19_DOB[0]": "03/15/1988",
    "Line7_AlienNumber[0]": "123456789",
    "Line18a_CityTownOfBirth[0]": "Mexico City"
}

validation_result = validate_required_fields("test_session_123", complete_answers)
print("Complete answers validation:")
print(json.dumps(validation_result, indent=2))

# Test case 2: Missing required fields
incomplete_answers = {
    "Line1a_FamilyName[0]": "Smith",
    "Line7_AlienNumber[0]": "987654321"
}

validation_result2 = validate_required_fields("test_session_123", incomplete_answers)
print("\nIncomplete answers validation:")
print(json.dumps(validation_result2, indent=2))


=== Testing Validation Function ===
Complete answers validation:
{
  "valid": true,
  "missing_fields": [],
  "completion_percentage": 60.0,
  "total_fields": 10,
  "filled_fields": 6
}

Incomplete answers validation:
{
  "valid": false,
  "missing_fields": [
    "Line1b_GivenName[0]",
    "Line19_DOB[0]"
  ],
  "completion_percentage": 20.0,
  "total_fields": 10,
  "filled_fields": 2
}


In [26]:
print("\n=== Testing PDF Form Filling ===")


input_pdf_path = "static/forms/I-765/form.pdf"  
output_pdf_path = "output/filled_pdfs/test_filled_form.pdf"

# Sample data matching your field structure
test_data = {
    "Line1a_FamilyName[0]": "TestLastName",
    "Line1b_GivenName[0]": "TestFirstName", 
    "Line1c_MiddleName[0]": "TestMiddleName",
    "Line2a_FamilyName[0]": "OtherLastName",
    "Line2b_GivenName[0]": "OtherFirstName",
    "Line2c_MiddleName[0]": "OtherMiddleName",
    "Line7_AlienNumber[0]": "A123456789",
    "Line19_DOB[0]": "01/15/1990",
    "Line18a_CityTownOfBirth[0]": "Test City"
}

# Check if input PDF exists
if os.path.exists(input_pdf_path):
    print(f"✓ Found PDF at: {input_pdf_path}")
    
    # Fill the PDF
    success = fill_pdf_form(input_pdf_path, output_pdf_path, test_data)
    
    if success:
        print(f"✓ PDF filled successfully!")
        print(f"✓ Output saved to: {output_pdf_path}")
        
        # Verify the output file exists
        if os.path.exists(output_pdf_path):
            file_size = os.path.getsize(output_pdf_path)
            print(f"✓ Output file size: {file_size} bytes")
            
            # Try to read the filled PDF to verify it's valid
            try:
                reader = PdfReader(output_pdf_path)
                print(f"✓ Filled PDF has {len(reader.pages)} pages")
                print(f"✓ PDF is readable and valid")
            except Exception as e:
                print(f"✗ Error reading filled PDF: {e}")
        else:
            print("✗ Output file was not created")
    else:
        print("✗ PDF filling failed")
        
else:
    print(f"✗ PDF not found at: {input_pdf_path}")
    print("Please update the input_pdf_path variable to point to your actual PDF file")


=== Testing PDF Form Filling ===
✓ Found PDF at: static/forms/I-765/form.pdf
✓ PDF filled successfully!
✓ Output saved to: output/filled_pdfs/test_filled_form.pdf
✓ Output file size: 319723 bytes
✓ Filled PDF has 7 pages
✓ PDF is readable and valid


In [27]:
print("\n=== File Download Information ===")

if os.path.exists(output_pdf_path):
    print(f"Your filled PDF is ready!")
    print(f"File location: {os.path.abspath(output_pdf_path)}")
    print(f"File size: {os.path.getsize(output_pdf_path)} bytes")
else:
    print("❌ No filled PDF was created. Check the errors above.")


=== File Download Information ===
Your filled PDF is ready!
File location: /Users/lac-phongnguyen/code/uc-berkeley-ai-hackathon/backend/output/filled_pdfs/test_filled_form.pdf
File size: 319723 bytes
